# Training a Neural Network (Basic)

- Creat a model.
- Choose a appropriate loss function.
- Create a dataset.
- Define a optimizer.
- Run a training Loop.
    - calculate loss (forward pass).
    - Calculate gradients.
    - Update the model parameters.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader

In [3]:
seed = 40

# 1. Dataset

In [4]:
df = pd.read_csv('datasets/salary_data_clean.csv')
df.head()

,age,gender,education,job_title,experience,salary
0,32.0,1,0,159,5.0,90000.0
1,28.0,0,1,17,3.0,65000.0
2,45.0,1,2,130,15.0,150000.0
3,36.0,0,0,101,7.0,60000.0
4,52.0,1,1,22,20.0,200000.0


In [5]:
feature_cols = ['age', 'experience']
target_cols = ['salary']
inputs = df[feature_cols].values
targets = df[target_cols].values

In [6]:
# input_tensor = torch.tensor(inputs).float()
input_tensor = torch.tensor(inputs, dtype=torch.float32)

In [7]:
# target_tensor = torch.tensor(targets).float()
target_tensor = torch.tensor(targets, dtype=torch.float32)

In [8]:
input_tensor.shape, target_tensor.shape

(torch.Size([373, 2]), torch.Size([373, 1]))

In [9]:
dataset = TensorDataset(input_tensor, target_tensor)
dataset

In [10]:
# Split the dataset.
from torch.utils.data import DataLoader, TensorDataset, random_split

train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size =  len(dataset) - train_size - val_size

In [11]:
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])
train_set

In [12]:
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=True)

In [13]:
# for batch_input, batch_target in dataloader:
#     print(batch_input.shape, batch_target.shape)

# 2. Model

In [14]:
model = nn.Sequential(
    nn.Linear(in_features=input_tensor.shape[-1], out_features=10),
    nn.ReLU(),
    nn.Linear(in_features=10, out_features=5),
    nn.ReLU(),
    nn.Dropout(p=0.5), # Dropout layer: regularization. p=0.5 => set 50% of output neurons to zero.
    nn.Linear(in_features=5, out_features=5),
    nn.ReLU(),
    nn.Linear(in_features=5, out_features=1)
)

In [15]:
# model(batch_input)

# 3. Loss Function

In [16]:
criterion = nn.MSELoss()

# 4. Optimizer

In [17]:
# With Weight decay regularizaiton
# weight decay will add penalty term to the loss function to make the weight params small.
# higher value => less likely to overfit.
# optimizer = optim.SGD(model.parameters(), lr=0.005, weight_decay=1e-4) 

# Without Weight Decay regularization
optimizer = optim.SGD(model.parameters(), lr=0.005)

# 5. Training Loop

In [18]:
# for val_batch_input, val_batch_target in val_loader:
#     break;

# model.eval()
# with torch.no_grad():
#     yhat = model(val_batch_input)
#     print(yhat)

In [19]:
# !pip3 install torchmetrics

In [34]:
num_epoch = 10

for epoch in range(num_epoch):

    model.train()

    total_loss = 0
    for batch_input, batch_target in train_loader:

        optimizer.zero_grad()

        y_hat = model(batch_input)

        loss = criterion(y_hat, batch_target)
        loss.backward()
        total_loss += loss.item()        

        optimizer.step()

    validation_loss = 0

    model.eval()

    with torch.no_grad():
        for val_batch_input, val_batch_target in val_loader:
            y_hat = model(val_batch_input)
            # print(y_hat)

            loss = criterion(y_hat, val_batch_target)
            validation_loss += loss.item()  

    avg_val_loss = validation_loss/len(val_loader)
    print(f"Epoch {epoch}: Train Loss: {total_loss/len(train_loader):.2f} | Val Loss: {avg_val_loss:.2f}")
        
    if epoch % 100 == 0 and epoch != 0:
        # print(f"Epoch {epoch}: Train Loss: {total_loss/len(train_loader):.2f} | Val Loss: {avg_val_loss:.2f}")
        print(f"Epoch {epoch}: Train Loss: {total_loss/len(train_loader):.2f}")
        # print(model[0].weight)

Epoch 0: Train Loss: 5671784755.20 | Val Loss: 4527600896.00
Epoch 1: Train Loss: 4914308659.20 | Val Loss: 3137552768.00
Epoch 2: Train Loss: 4483439180.80 | Val Loss: 3494546304.00
Epoch 3: Train Loss: 4044477696.00 | Val Loss: 3469208960.00
Epoch 4: Train Loss: 3831949644.80 | Val Loss: 1823806496.00
Epoch 5: Train Loss: 3565278822.40 | Val Loss: 2945678592.00
Epoch 6: Train Loss: 3384737292.80 | Val Loss: 2677851392.00
Epoch 7: Train Loss: 3064547008.00 | Val Loss: 5064065408.00
Epoch 8: Train Loss: 2936999654.40 | Val Loss: 2767123072.00
Epoch 9: Train Loss: 2728264012.80 | Val Loss: 2498087808.00
